In [1]:
import cv2
import xml.etree
import numpy as np
import pathlib
from pathlib import Path
import os
import glob
import xml.etree.ElementTree as ET

In [9]:
# segment the images and write to raw in some appropriate structure, one volume at a time
def write_imgs_gts(path_to_volume):
  
    path_to_imgs = os.path.join(path_to_volume, 'Images')
    path_to_page = os.path.join(path_to_volume, 'page')

    imgs = glob.glob1(path_to_imgs, '*.jpg')
    page_xmls = glob.glob1(path_to_page, '*.xml')
    imgs.sort()
    page_xmls.sort()
    
    imgs = [os.path.join(path_to_imgs, img) for img in imgs]
    page_xmls = [os.path.join(path_to_page, page) for page in page_xmls]
    ground_truths = []
    assert len(imgs) == len(page_xmls)

    print(imgs[0])

    os.mkdir(os.path.join(path_to_volume, 'Line_Images'))

    total = 0

    for image, page in zip(imgs, page_xmls):

        checksum = 0

        total += 1

        print(total)

        page_dir = os.path.join(path_to_volume, 'Line_Images', Path(image).name.split('.')[0])

        os.mkdir(page_dir)
        
        tree = ET.parse(page)
        root=tree.getroot()    
        
        img = cv2.imread(image)
        dst = cv2.fastNlMeansDenoising(img, h=31, templateWindowSize=7, searchWindowSize=21)
        img_grey = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.medianBlur(img_grey,3).astype('uint8')
        img = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        #img_gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
        #ret, img = cv2.threshold(img_gray, 145, 255, cv2.THRESH_BINARY)

        for text_line in root.iter('{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextLine'):

            line_img_file_path = os.path.join(page_dir, Path(path_to_volume).name + '_'  + Path(page_dir).name + '_' + str(checksum).zfill(4) + '.jpg')
        
            empty_text_field = False
            
            for child in text_line:
                
                if child.tag == '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextEquiv':
                    for text_field in child:
                        if text_field.text == None or text_field.text == '':
                            empty_text_field = True
                            continue
                        ground_truths.append(line_img_file_path + '\t' + text_field.text)

                elif child.tag == '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Coords':
                
                    mask = np.zeros(img.shape[0:2], dtype=np.uint8)
                    
                    coordinates = child.attrib['points'].split()
                    temp = [coord.split(',') for coord in coordinates]
                    temp2 = [[int(x) for x in lst] for lst in temp]
                    points = np.array(temp2)
                    
                    cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)
                    res = cv2.bitwise_and(img,img,mask = mask)
                    rect = cv2.boundingRect(points)

                    wbg = np.ones_like(img, np.uint8)*255
                    cv2.bitwise_not(wbg,wbg, mask=mask)
                    # overlap the resulted cropped image on the white background
                    dst = wbg+res
                
                    cropped = dst[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
                    #cropped_border = cv2.copyMakeBorder(src=cropped, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255),)                   

            if not empty_text_field:
                checksum +=1
                #img_file_path = os.path.join(page_dir, Path(path_to_volume).name + '_'  + Path(page_dir).name + '_' + str(checksum).zfill(4) + '.jpg')
                cv2.imwrite(line_img_file_path, cropped) 
        
        #assert checksum == len(ground_truths)
        
        path_to_ground_truths = os.path.join(path_to_volume, 'gt.txt')
        with open(path_to_ground_truths, 'w') as f:
            f.write('\n'.join(ground_truths))

In [10]:
write_imgs_gts('/home/erik/Riksarkivet/Projects/handwritten-text-recognition/raw/Gothenburg_police_records_pre_1900/A_II__12')

/home/erik/Riksarkivet/Projects/handwritten-text-recognition/raw/Gothenburg_police_records_pre_1900/A_II__12/Images/333686_0001.jpg
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
